# Hw7

In [1]:
from surprise import SVD, SVDpp, NMF, Dataset
import pandas as pd

In [2]:
data = Dataset.load_builtin('ml-100k')
df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


## Підбір параметрів з використанням кросс-валідації за допомогою GridSearchCV

### SVD

In [3]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors': [10, 50, 100], 'n_epochs': [5, 10, 20], 'lr_all': [0.001, 0.005, 0.01], 'reg_all': [0.2, 0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1)
gs.fit(data)


In [4]:
print(f" RMSE best score: {gs.best_score['rmse']}")
print(f" MAE best score: {gs.best_score['mae']}")
print(f" SVD best params: \n{gs.best_params}")

 RMSE best score: 0.9462062560679203
 MAE best score: 0.7531278549303168
 SVD best params: 
{'rmse': {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.2}, 'mae': {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.2}}


### SVD++

In [5]:
gs_svd_pp = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1)
gs_svd_pp.fit(data)

In [6]:
print( f" RMSE best score: {gs_svd_pp.best_score['rmse']}")
print(f" MAE best score: {gs_svd_pp.best_score['mae']}")
print(f"\n SVD++ best params: \n{gs_svd_pp.best_params}")

 RMSE best score: 0.9438813435677669
 MAE best score: 0.7508917176160145

 SVD++ best params: 
{'rmse': {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.2}, 'mae': {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.2}}


### NMF

In [7]:
param_grid_nmf = {'n_factors': [10, 50, 100], 'n_epochs': [5, 10, 20], 'reg_pu': [0.2, 0.4, 0.6], 'reg_qi': [0.2, 0.4, 0.6]}
gs_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse', 'mae'], cv=3, n_jobs=-1)
gs_nmf.fit(data)        


In [8]:
print(f" RMSE best score: {gs_nmf.best_score['rmse']}")
print(f" MAE best score: {gs_nmf.best_score['mae']}")
print(f" NMF best params: \n{gs_nmf.best_params}")

 RMSE best score: 0.9525871594036149
 MAE best score: 0.7465255842987881
 NMF best params: 
{'rmse': {'n_factors': 100, 'n_epochs': 10, 'reg_pu': 0.2, 'reg_qi': 0.6}, 'mae': {'n_factors': 100, 'n_epochs': 20, 'reg_pu': 0.2, 'reg_qi': 0.2}}


## Будуємо тренувальну та тестову вибірку

In [12]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

## Будуємо та тренуємо модель SVD з найкращими параметрами

In [9]:
model_svd = gs.best_estimator['rmse']
model_svd.fit(trainset)


In [13]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [14]:
from surprise import get_dataset_dir
def read_item_names():
    file_name = get_dataset_dir() + "/ml-100k/ml-100k/u.item"
    rid_to_name = {}
    name_to_rid = {}
    with open(file_name, encoding="ISO-8859-1") as f:
        for line in f:
            line = line.split("|")
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

rid_to_name, name_to_rid = read_item_names()

### Отримуємо рекомендації за допомогою SVD

In [ ]:
predictions = model_svd.test(testset)

top_n_svd = get_top_n(predictions, n=10)

### Виводимо рекомендації для користувача за user_id

In [56]:
for n in top_n_svd['936']:
    print(rid_to_name[n[0]], n[1].round(2))

Pather Panchali (1955) 4.75
Close Shave, A (1995) 4.7
Wrong Trousers, The (1993) 4.67
Schindler's List (1993) 4.65
Shawshank Redemption, The (1994) 4.62
Wallace & Gromit: The Best of Aardman Animation (1996) 4.57
Usual Suspects, The (1995) 4.57
Casablanca (1942) 4.56
Rear Window (1954) 4.53
12 Angry Men (1957) 4.53


## Будуємо та тренуємо модель SVD++ з найкращими параметрами

In [ ]:
model_svdpp = gs_svd_pp.best_estimator['rmse']
model_svdpp.fit(data.build_full_trainset())

### Отримуємо рекомендації за допомогою SVD++

In [62]:
predictions_svdpp = model_svdpp.test(testset)
top_n_svdpp = get_top_n(predictions_svdpp, n=10)

### Виводимо рекомендації для користувача за user_id

In [66]:
for n in top_n_svdpp['936']:
    print(rid_to_name[n[0]], n[1].round(2))

Pather Panchali (1955) 4.73
Close Shave, A (1995) 4.72
Wrong Trousers, The (1993) 4.69
Schindler's List (1993) 4.65
Shawshank Redemption, The (1994) 4.62
Wallace & Gromit: The Best of Aardman Animation (1996) 4.59
Usual Suspects, The (1995) 4.58
Casablanca (1942) 4.58
Rear Window (1954) 4.55
12 Angry Men (1957) 4.55


## Будуємо та тренуємо модель NMF з найкращими параметрами

In [67]:
model_nmf = gs_nmf.best_estimator['rmse']
model_nmf.fit(data.build_full_trainset())

### Отримуємо рекомендації за допомогою NMF

In [68]:
predictions_nmf = model_nmf.test(testset)
top_n_nmf = get_top_n(predictions_nmf, n=10)

### Виводимо рекомендації для користувача за user_id

In [72]:
for n in top_n_nmf['936']:
    print(rid_to_name[n[0]], round(float(n[1]), 2))

Santa with Muscles (1996) 5.0
Great Day in Harlem, A (1994) 5.0
Pather Panchali (1955) 5.0
Aiqing wansui (1994) 5.0
Someone Else's America (1995) 5.0
Some Mother's Son (1996) 5.0
Saint of Fort Washington, The (1993) 4.99
Faust (1994) 4.95
Anna (1996) 4.92
Close Shave, A (1995) 4.9


## Висновки:
Моделі SVD та SVD++ дають схожі результати, але SVD++ занадто ресурсоємний. Натомість NMF з точки зору швидкодії має накращі показники, але результати суттєво відрізняються від SVD++ та SVD. 